In [216]:
# Traffic prediction ML project
# Imports
import pandas as pd
import numpy as np


In [217]:
'''
Feature engineering notes:
    - Time and date
    - Combine Vehicle continuing direction and vehicle going direction. Oncoming/incoming or same direction
    - The weather and surface cond columns may help each other...
    - Modify Light so that we replace 'Dark -- Unknown Lighting' with 'Dark'
    
'''

"\nFeature engineering notes:\n    - Time and date\n    - Combine Vehicle continuing direction and vehicle going direction. Oncoming/incoming or same direction\n    - The weather and surface cond columns may help each other...\n    - Modify Light so that we replace 'Dark -- Unknown Lighting' with 'Dark'\n    \n"

In [218]:
data = pd.read_csv("traffic.csv", na_values=['Unknown', 'None', 'UNKNOWN'])
# data.shape - 172,105 rows, 43 columns
# data['Off-Road Description'].unique() # Determine that the values in this column is not important
# data['Driverless Vehicle'].unique() # No driverless vehicles, so column is unneeded
# data['Parked Vehicle'].value_counts() # 2,600 parked vehicles got an in accident. Might be interesting to see if there's a correlation between that and the severity of the accident.
# data['Related Non-Motorist'].value_counts() - There's only about 5000 fields here, it may be too little to make a discernible impact on the dataset.




# Look into what the cross street and route type means
# DROP Report Number, Local Case Number, Off-road description, Road name, Cross-street name, Driverless Vehicle, Municipality (many missing values),  Related Non-Motorist, Non-Motorist Substance Abuse, Person ID, Circumstance (??), Vehicle ID, Vehicle Make, Vehicle Model, Equiptment Problems, Location (as it combines Lat & Long)

/var/folders/xp/ph5sj5y90_j0747lx52mp81r0000gn/T/ipykernel_36999/3591236047.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("traffic.csv", na_values=['Unknown', 'None', 'UNKNOWN'])


In [219]:
data['Collision Type'].value_counts()
# 12% other, 717 unknown. We may want to use similar-case imputation

data['Non-Motorist Substance Abuse'].value_counts()
# There are only ~200 data points that actually had substance abuse, so this should be dropped.

data['Driver Distracted By'].value_counts()
# 1/5th of the dataset is missing, over half has drivers that were not distracted. Maybe there's a correlation between distracted driving and severity? Maybe we can train a subset of the data on this to reveal more interesting patterns. Or, we could inject and say that half of the missing is distracted and the other half is not distracted. Or, maybe we should just drop it.

data['Drivers License State'].value_counts()
# correlation between driving out of state lines and being at fault? are drivers closer to home truly more likely to get in an accident? Outliers - FM, NS, MP, SK, etc.

data['Vehicle Movement'].value_counts()
# We need to get creative with our encoding, such as moving quickly, moving at a slower rate, or manuvering (parking, passing, turning, etc)

# This is our class label. We need to OHC different categories.
data['Injury Severity'].value_counts()
# the majority of the dataset isn't injured, fortunately.
# we will just keep the 5 predictors
    # No injury
    # Possible
    # Minor
    # Serious
    # Fatal :(

Injury Severity
NO APPARENT INJURY          141185
POSSIBLE INJURY              17482
SUSPECTED MINOR INJURY       11870
SUSPECTED SERIOUS INJURY      1415
FATAL INJURY                   153
Name: count, dtype: int64

In [220]:
data['Vehicle Dir Changed'] = np.where(data['Vehicle Continuing Dir'] == data['Vehicle Going Dir'], 'No', 'Yes')
data['Vehicle Multiple Impacts'] = np.where(data['Vehicle First Impact Location'] == data['Vehicle Second Impact Location'], 'No', 'Yes')
data = data.drop(columns = ['Agency Name', 'Report Number', 'Local Case Number', 'Off-Road Description', 'Road Name', 'Cross-Street Name', 'Driverless Vehicle', 'Municipality', 'Non-Motorist Substance Abuse', 'Person ID', 'Circumstance', 'Vehicle ID', 'Vehicle Make', 'Vehicle Model', 'Equipment Problems', 'Location','Driverless Vehicle', 'Vehicle Continuing Dir', 'Vehicle Going Dir', 'Route Type', 'Vehicle First Impact Location', 'Vehicle Second Impact Location', 'Related Non-Motorist'], axis = 1)
data.head()

,ACRS Report Type,Crash Date/Time,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Driver At Fault,...,Vehicle Damage Extent,Vehicle Body Type,Vehicle Movement,Speed Limit,Parked Vehicle,Vehicle Year,Latitude,Longitude,Vehicle Dir Changed,Vehicle Multiple Impacts
0,Property Damage Crash,05/31/2019 03:00:00 PM,NaN,OTHER,CLEAR,NaN,DAYLIGHT,NaN,NaN,Yes,...,SUPERFICIAL,PASSENGER CAR,PARKING,15,No,2004,39.150044,-77.063089,No,No
1,Property Damage Crash,07/21/2023 05:59:00 PM,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,No,...,DISABLING,PASSENGER CAR,MAKING LEFT TURN,40,No,2011,39.159264,-77.219025,Yes,Yes
2,Property Damage Crash,07/20/2023 03:10:00 PM,Maryland (State),STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,Yes,...,FUNCTIONAL,PICKUP TRUCK,ACCELERATING,35,No,2019,39.109535,-77.075806,No,No
3,Property Damage Crash,07/23/2023 12:10:00 PM,County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,Yes,...,FUNCTIONAL,PASSENGER CAR,STARTING FROM LANE,40,No,2016,39.190149,-77.266766,No,No
4,Property Damage Crash,07/24/2023 06:10:00 AM,County,HEAD ON LEFT TURN,CLOUDY,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,Yes,...,FUNCTIONAL,(SPORT) UTILITY VEHICLE,MAKING LEFT TURN,35,No,2016,39.172558,-77.203745,Yes,No


In [221]:
data.loc[data['Weather'] == 'RAINING', 'Light'] = data.loc[data['Weather'] == 'RAINING', 'Light'].fillna('DARK')
data.loc[data['Weather'] == 'FOGGY', 'Light'] = data.loc[data['Weather'] == 'FOGGY', 'Light'].fillna('DARK')


# Dealing with Date/Time column
# Monday is 0, so SUNDAY is 6
import datetime
new_weekdays = []
pos = 0
count = 0
for i, date in data['Crash Date/Time'].items() : 
    date_parsed = date.split(' ')
    time = date_parsed[0].split('/')
    obj = datetime.datetime(int(time[2]), int(time[0]), int(time[1]))
    new_weekdays.append(obj.weekday())
    hour = int(date_parsed[1].split(':')[0])

    if pd.isnull(data['Light'][i]) :
        if (hour < 5 or hour > 22) : 
            data['Light'][i] = 'DARK'
        elif (hour >= 5 and hour <= 8): 
            data['Light'][i] = 'DAWN' 
        elif (hour > 8 and hour <= 15) : 
            data['Light'][i] = 'LIGHT'
        elif (hour > 15 and hour <= 22) : 
            data['Light'][i] = 'DUSK'
            
data['Crash Date/Time'] = new_weekdays
df_encoded = pd.get_dummies(data['Crash Date/Time'], columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
# for num in range(len(days)) :
#     df_encoded[days[num]] = new_weekdays[num]
data = pd.concat([data, df_encoded] , axis=1)
data = data.drop(columns=['Crash Date/Time'])
data.head()


/var/folders/xp/ph5sj5y90_j0747lx52mp81r0000gn/T/ipykernel_36999/1882687334.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Light'][i] = 'DARK'
/var/folders/xp/ph5sj5y90_j0747lx52mp81r0000gn/T/ipykernel_36999/1882687334.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Light'][i] = 'DARK'
/var/folders/xp/ph5sj5y90_j0747lx52mp81r0000gn/T/ipykernel_36999/1882687334.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

,ACRS Report Type,Cross-Street Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Driver At Fault,Injury Severity,...,Longitude,Vehicle Dir Changed,Vehicle Multiple Impacts,0,1,2,3,4,5,6
0,Property Damage Crash,NaN,OTHER,CLEAR,NaN,DAYLIGHT,NaN,NaN,Yes,NO APPARENT INJURY,...,-77.063089,No,No,False,False,False,False,True,False,False
1,Property Damage Crash,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,No,NO APPARENT INJURY,...,-77.219025,Yes,Yes,False,False,False,False,True,False,False
2,Property Damage Crash,Maryland (State),STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,Yes,NO APPARENT INJURY,...,-77.075806,No,No,False,False,False,True,False,False,False
3,Property Damage Crash,County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,Yes,NO APPARENT INJURY,...,-77.266766,No,No,False,False,False,False,False,False,True
4,Property Damage Crash,County,HEAD ON LEFT TURN,CLOUDY,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,Yes,NO APPARENT INJURY,...,-77.203745,Yes,No,True,False,False,False,False,False,False


In [222]:
data.loc[data['Weather'] == 'CLEAR', 'Surface Condition'] = data.loc[data['Weather'] == 'CLEAR', 'Surface Condition'].fillna('DRY')
data.loc[data['Weather'] == 'RAINING', 'Surface Condition'] = data.loc[data['Weather'] == 'RAINING', 'Surface Condition'].fillna('WET')



data['Surface Condition'].value_counts()
data = data.drop(columns=['Weather'])



In [223]:
# Map out ACRS Report Type to numerical 
data['ACRS Report Type'] = data['ACRS Report Type'].map({'Property Damage Crash': 0, 'Injury Crash': 1, 'Fatal Crash': 2})
data['Vehicle Changed Dir'] = data['Vehicle Changed Dir'].map({'No': 0, 'Yes': 1})
data['Vehicle Multiple Impacts'] = data['Vehicle Multiple Impacts'].map({'No': 0, 'Yes': 1})


data['ACRS Report Type'].value_counts()

KeyError: 'Vehicle Changed Dir'

In [ ]:
data['Surface Condition'].value_counts()

Surface Condition
DRY                       120569
WET                        28426
ICE                         1058
SNOW                         938
UNKNOWN                      509
SLUSH                        204
OTHER                        164
MUD, DIRT, GRAVEL             46
WATER(STANDING/MOVING)        41
OIL                           28
SAND                           4
Name: count, dtype: int64

In [ ]:
data['Weather'].value_counts()
data['Weather'] = data['Weather'].map({'DRY': , 'Injury Crash': 1, 'Fatal Crash': 2})


Weather
CLEAR                       116774
RAINING                      20608
CLOUDY                       17294
SNOW                          1444
UNKNOWN                        692
FOGGY                          676
WINTRY MIX                     391
OTHER                          370
SLEET                          218
SEVERE WINDS                   154
BLOWING SNOW                   115
BLOWING SAND, SOIL, DIRT        15
Name: count, dtype: int64

In [ ]:
print(data.columns)

Index([        'ACRS Report Type',        'Cross-Street Type',
                 'Collision Type',                  'Weather',
              'Surface Condition',                    'Light',
                'Traffic Control',   'Driver Substance Abuse',
                'Driver At Fault',          'Injury Severity',
           'Driver Distracted By',    'Drivers License State',
          'Vehicle Damage Extent',        'Vehicle Body Type',
               'Vehicle Movement',              'Speed Limit',
                 'Parked Vehicle',             'Vehicle Year',
                       'Latitude',                'Longitude',
            'Vehicle Dir Changed', 'Vehicle Multiple Impacts',
                                0,                          1,
                                2,                          3,
                                4,                          5,
                                6],
      dtype='object')


In [225]:
# data.loc[data['Weather'] == 'CLEAR', 'Surface Condition'] = data.loc[data['Weather'] == 'CLEAR', 'Surface Condition'].fillna('DRY')

# We are assuming that since the officer did not record any alcohol or drugs, 
# that the driver was not under the influence.
data['Driver Substance Abuse'].fillna('NONE DETECTED', inplace=True)
data['Driver Substance Abuse'].fillna('NONE DETECTED', inplace=True)
WHERE substance abuse is alcohol contributed set it to alcohol detected 
data['Driver Substance Abuse'].loc('')


print(data['Driver Substance Abuse'].isna().value_counts())
print(data['Driver Substance Abuse'].value_counts())

# If the cop found drugs or alcohol, he probably would've written it down.
# We are assuming the empty values are None


# Alcohol
# Illicit Drug
# Medication
# None Present

Driver Substance Abuse
False    172105
Name: count, dtype: int64
Driver Substance Abuse
NONE DETECTED                 165849
ALCOHOL PRESENT                 4084
ALCOHOL CONTRIBUTED             1435
ILLEGAL DRUG PRESENT             258
MEDICATION PRESENT               117
ILLEGAL DRUG CONTRIBUTED         102
COMBINED SUBSTANCE PRESENT        92
MEDICATION CONTRIBUTED            64
OTHER                             58
COMBINATION CONTRIBUTED           46
Name: count, dtype: int64
